In [138]:
import pandas as pd
import os
from cdcqr.common.utils import LOCAL_DATA_DIR, dict_hash
from cdcqr.data.glassnode.features.feature_loaders import produce_feature_BTCSSR
from cdcqr.backtest.vbt.sample_params import param_dict
import json
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
import cdcqr.backtest.vbt.runsignal as runsig
from importlib import reload

In [8]:
# load price
price = pd.read_pickle(os.path.join(LOCAL_DATA_DIR,'BTC-PERP@ftx.pickle'))

#load feature
asset = 'BTC'
feature = 'BTCSSR'
signalid = 'long-BTCSSR'
feature_data_fname = produce_feature_BTCSSR()
feature_data_dir = os.path.join(LOCAL_DATA_DIR, 'feature_data', '{}.pickle'.format(feature_data_fname))
df4a = pd.read_pickle(feature_data_dir)


In [15]:
display(price.memory_usage().sum()/10e6)
display(df4a.memory_usage().sum()/10e6)

2.2308336

0.9082

In [16]:
asset = 'LINK'
price0 = pd.read_pickle('/core/tmp/ppt_prc_{}_1m_20210101_20220228.pkl'.format(asset))
liq0 = pd.read_pickle('/core/tmp/ppt_liqb_{}_1m_20210101_20220228.pkl'.format(asset))
df = pd.read_pickle('/core/tmp/glass_balexch_{}_10m_20210101_20220131.pickle'.format(asset))

display(price0.memory_usage().sum()/10e6)
display(liq0.memory_usage().sum()/10e6)
display(df.memory_usage().sum()/10e6)

3.5311248

0.1773456

0.1203136

In [26]:
def getsignal3(glass, price0, liq0, qtl=0.99, liql=20, priceq=0.0, qtl_w=0):
    glass['99q'] = glass['v'].rolling(60*24).quantile(qtl)
    glass['signal'] = (glass['v']>=glass['99q'])
    price0['glass_signal'] = glass['signal']
    price0['glass_signal'] = price0['glass_signal'].fillna(False)
    
    liq = liq0.resample("1Min").last().fillna(0)
    liq['spikes'] = liq[liq.v>liql]['v']
    liq['spikes'] = liq['spikes'].fillna(0).rolling(100).max().fillna(0)
    price0['liq_signal'] = liq['spikes'][liq['spikes']>liql]
    price0['liq_signal'] = price0['liq_signal'].fillna(False)
    
    if qtl_w == 0:
        price0['cq_signal'] = True
    else:
        price0['cq'] = price0['c'].rolling(qtl_w).quantile(priceq).resample("1Min").last().ffill()
        price0['cq_signal'] = (price0['c'] >= price0['cq']).fillna(False)

    price0['s'] = price0['glass_signal'] & price0['liq_signal'] & price0['cq_signal']
    return price0[['c','glass_signal','liq_signal','cq_signal','s']]

In [150]:
asset='AAVE'
signalid = 'short-{}liq'.format(asset)

price0 = pd.read_pickle('/core/tmp/ppt_prc_{}_1m_20210101_20220228.pkl'.format(asset))
liq0 = pd.read_pickle('/core/tmp/ppt_liqb_{}_1m_20210101_20220228.pkl'.format(asset))

df = pd.read_pickle('/core/tmp/glass_balexch_{}_10m_20210101_20220131.pickle'.format(asset))
glass = df.copy().resample("1Min").last().fillna(0)
glass.columns = ['v']

param_dict_raw = param_dict.copy()
qtl_params = [0.995, 0.99, 0.975]
liql_params =  [5, 10, 20, 50]
priceq_params = [0]
qtlw_params = [0]

test=True
if test:
    tp_params = [3, 5] # 
    sl_params = [1, 2] # ts = sl 
    n1_params = [1, 10]
    n2_params = [101]
    rsil_params = [10]
    rsilag_params = [14]
    maf_params = [5]
    mas_params = [15]
else:
    tp_params = [3, 5, 7, 10] # 
    sl_params = [1, 2, 3 , 5] # ts = sl 
    n1_params = [None]
    n2_params = [None]
    rsil_params = [30] # 2 runs with rsil rsilag
    rsilag_params = [14]
    maf_params = [5, 60] # 4 runs for maf/mas
    mas_params = [100, 200]

backtest_info = {}
backtest_info['asset'] = asset
backtest_info['signalid'] = signalid
backtest_info['side'] = 'short'
backtest_info['qtl_params'] = qtl_params
backtest_info['priceq_params'] = priceq_params
backtest_info['qtlw_params'] = qtlw_params
backtest_info['liql_params'] = liql_params
backtest_info['tp_params'] = tp_params
backtest_info['sl_params'] = sl_params
backtest_info['n1_params'] = n1_params
backtest_info['n2_params'] = n2_params

backtest_info['rsil_params'] = rsil_params
backtest_info['rsilag_params'] = rsilag_params
backtest_info['maf_params'] = maf_params
backtest_info['mas_params'] = mas_params


backtest_info_hashed = dict_hash(backtest_info)
backtest_job_info = '{}_{}'.format(json.dumps(backtest_info), backtest_info_hashed)
job_start_log_info = 'Starting backtest job {}'.format(backtest_job_info)

backtest_configs = []
price = price0
for qtl in qtl_params:
    for liql in liql_params:
        for priceq in priceq_params:
            for qtlw in qtlw_params:
                #print(qtl, liql, priceq, qtlw)
                signal = getsignal3(glass, price, liq0, qtl,liql,priceq,qtlw)['s']
                for tp in tp_params:
                    for sl in sl_params:
                        for n1 in n1_params:
                            for n2 in n2_params:
                                for rsil in rsil_params:
                                    rsih = 100-rsil
                                    for maf in maf_params:
                                        for mas in mas_params:
                                            for railag in rsilag_params:
                                                param_dict = param_dict_raw.copy()
                                                param_dict['qtl'] = qtl
                                                param_dict['liql'] = liql
                                                param_dict['priceq'] = priceq
                                                param_dict['qtlw'] = qtlw
                                                param_dict['price'] = price
                                                param_dict['signal'] = signal
                                                param_dict['qtl'] = qtl
                                                param_dict['tp'] = tp
                                                param_dict['sl'] = sl
                                                param_dict['n1'] = n1
                                                param_dict['n2'] = n2
                                                param_dict['rsil'] = rsil
                                                param_dict['rsih'] = rsih
                                                param_dict['maf'] = maf
                                                param_dict['mas'] = mas
                                                param_dict['railag'] = railag

                                                backtest_configs.append(param_dict)


In [151]:
param_dict1 = backtest_configs[0]
param_dict1['short']=True

In [155]:
reload(runsig)
res = runsig.wrapped_runsignal(param_dict1)

In [158]:
pd.DataFrame(res)

,tp,sl,n1,n2,qtl,lbw,rsil,rsih,rsilag,maf,...,side,liql,priceq,qtlw,railag,ts,k,sr,tr,ntrades
0,3,1,1,101,0.995,4000,10,90,14,5,...,long,5,0,0,14,1,ma_ma_pf,-0.800323,-0.135528,3.397849
1,3,1,1,101,0.995,4000,10,90,14,5,...,long,5,0,0,14,1,ma_ts_pf,0.075478,0.003040,3.268817
2,3,1,1,101,0.995,4000,10,90,14,5,...,long,5,0,0,14,1,ma_tpts_pf,-0.001583,-0.008118,3.268817
3,3,1,1,101,0.995,4000,10,90,14,5,...,long,5,0,0,14,1,ma_tpsl_pf,1.334210,0.371059,3.053763
4,3,1,1,101,0.995,4000,10,90,14,5,...,long,5,0,0,14,1,ma_n_pf,0.258026,0.046223,2.795699
5,3,1,1,101,0.995,4000,10,90,14,5,...,long,5,0,0,14,1,rsi_rsi_pf,-0.832841,-0.166355,0.086022
6,3,1,1,101,0.995,4000,10,90,14,5,...,long,5,0,0,14,1,rsi_ts_pf,-0.981534,-0.022155,0.086022
7,3,1,1,101,0.995,4000,10,90,14,5,...,long,5,0,0,14,1,rsi_tpts_pf,-0.981534,-0.022155,0.086022
8,3,1,1,101,0.995,4000,10,90,14,5,...,long,5,0,0,14,1,rsi_tpsl_pf,-1.642985,-0.044822,0.086022
9,3,1,1,101,0.995,4000,10,90,14,5,...,long,5,0,0,14,1,rsi_n_pf,-0.163943,-0.014535,0.086022


In [31]:
backtest_configs[0]['price'].memory_usage().sum()/10e6

4.3403409

In [33]:
backtest_configs[0]['signal'].memory_usage()/10e6

0.6620859

In [41]:
from sys import getsizeof

dct = {'a': backtest_configs[0]['price'], 'b': backtest_configs[0]['signal']}

print(getsizeof(dct))

232


In [46]:
from sys import getsizeof

dct1 = {'a': 1}

print(getsizeof(dct1))

232


In [64]:
import sys

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [49]:
print(getsizeof(backtest_configs[3]))

1176


In [53]:
get_size(dct1)

310

In [88]:
get_size(dd)

3173

In [ ]:
dd['signal'].memory_usage()

In [83]:
dd['price'].memory_usage().sum()

43403409

In [86]:
del dd['signal']

In [89]:
dd = backtest_configs[3].copy()

### SSR

In [130]:
price = pd.read_pickle(os.path.join(LOCAL_DATA_DIR,'BTC-PERP@ftx.pickle'))

#load feature
asset = 'BTC'
feature = 'BTCSSR'
signalid = 'long-BTCSSR'
feature_data_fname = produce_feature_BTCSSR()
feature_data_dir = os.path.join(LOCAL_DATA_DIR, 'feature_data', '{}.pickle'.format(feature_data_fname))
df4a = pd.read_pickle(feature_data_dir)

param_dict_raw = param_dict.copy()
qtl_params = [0.995, 0.99, 0.975, 0.96]
lbw_params = [1000, 2000, 3000, 4000]

test=True
if test:
    tp_params = [3, 5] # 
    sl_params = [1, 2] # ts = sl 
    n1_params = [1, 10]
    n2_params = [101]
    rsil_params = [10]
    rsilag_params = [14]
    maf_params = [5]
    mas_params = [100, 200]
else:
    tp_params = [3, 5, 7, 10] # 
    sl_params = [1, 2, 3 , 5] # ts = sl 
    n1_params = [1, 10, 100]
    n2_params = [101, 200, 500, 1000, 2000]
    rsil_params = [10, 20] # 2 runs with rsil rsilag
    rsilag_params = [14]
    maf_params = [5, 60] # 4 runs for maf/mas
    mas_params = [100, 200]

backtest_info = {}
backtest_info['asset'] = asset
backtest_info['signalid'] = signalid
backtest_info['side'] = 'long'
backtest_info['qtl_params'] = qtl_params
backtest_info['lbw_params'] = lbw_params
backtest_info['tp_params'] = tp_params
backtest_info['sl_params'] = sl_params
backtest_info['n1_params'] = n1_params
backtest_info['n2_params'] = n2_params

backtest_info['rsil_params'] = rsil_params
backtest_info['rsilag_params'] = rsilag_params
backtest_info['maf_params'] = maf_params
backtest_info['mas_params'] = mas_params


backtest_info_hashed = dict_hash(backtest_info)
backtest_job_info = '{}_{}'.format(json.dumps(backtest_info), backtest_info_hashed)
job_start_log_info = 'Starting backtest job {}'.format(backtest_job_info)


backtest_configs = []
rsi_or_ma=['rsi', 'ma']
for lbw in lbw_params:
    feature_pctrank = '{}_pctrank{}'.format(feature, lbw)
    df4a[feature_pctrank] = df4a[feature].rolling(lbw).apply(pctrank)
    df4b = df4a[df4a.index>='20210101']
    for qtl in qtl_params:
        signal = df4b[feature_pctrank]>qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
        signal = signal.reindex(price.index).ffill()
        for tp in tp_params:
            for sl in sl_params:
                for n1 in n1_params:
                    for n2 in n2_params:
                        for rsil in rsil_params:
                            rsih = 100-rsil
                            for maf in maf_params:
                                for mas in mas_params:
                                    for railag in rsilag_params:

                                        param_dict = param_dict_raw.copy()
                                        param_dict['qtl'] = qtl
                                        param_dict['lbw'] = lbw
                                        param_dict['signal'] = signal
                                        param_dict['price'] = price
                                        param_dict['tp'] = tp
                                        param_dict['sl'] = sl
                                        param_dict['n1'] = n1
                                        param_dict['n2'] = n1 + n2
                                        param_dict['rsil'] = rsil
                                        param_dict['rsih'] = rsih
                                        param_dict['maf'] = maf
                                        param_dict['mas'] = mas
                                        param_dict['railag'] = railag

                                        backtest_configs.append(param_dict)

KeyboardInterrupt: 

In [129]:
(signal.index.max() - signal.index.min()).days/7

46.285714285714285

In [ ]:
backtest_configs

In [104]:
dd = backtest_configs[3].copy()
display(get_size(dd))
display(dd['price'].memory_usage().sum())
display(dd['signal'].memory_usage())

69554221

22308336

4182813

In [102]:
del dd['signal']
display(get_size(dd))
del dd['price']
display(get_size(dd))

68233661

22308336

4182813

69824558

2972

In [110]:
dd['price'].memory_usage().sum()

22308336

In [115]:
import numpy as np
dd['price'].astype(np.float32).memory_usage().sum()

13013196

In [120]:
dd['price'].astype(np.float16).head()

,o,h,l,c,v
exch_ts,,,,,
2021-01-01 00:00:00,28960.0,29008.0,28944.0,29008.0,103.500000
2021-01-01 00:01:00,29008.0,29056.0,29008.0,29056.0,43.031250
2021-01-01 00:02:00,29056.0,29056.0,29008.0,29024.0,95.125000
2021-01-01 00:03:00,29024.0,29040.0,29008.0,29024.0,10.046875
2021-01-01 00:04:00,29024.0,29024.0,29008.0,29008.0,22.406250


In [123]:
dd['price'].astype(np.float32).memory_usage().sum()

13013196

In [124]:
dd['price'].memory_usage().sum()

22308336